## Importing libraries and Loading data

In [7]:
import math, datetime, time, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import math, datetime, time, random
import matplotlib.pyplot as plt
import itertools
import tqdm

def load_data(data_path):
    data = pd.read_csv(data_path)  
    return data

inbound = load_data("inbound_loads.csv")
outbound = load_data("outbound_laods.csv")
weather = load_data("weather.csv")
#For loop to ensure that all pallet data is in the same dataframe
pallet = load_data("Pallet_history_Gold_Spike[0].csv")
for x in range(1, 10):
    pallet = pd.concat([pallet, load_data(f"Pallet_history_Gold_Spike[{x}].csv")])
pallet = pallet.drop(['lot_code', 
                      'tran_type', 
                      'final_pallet_code', 
                      'warehouse_facility_id',
                      'source_system_id'], axis=1)
trainentest = load_data("demand_kWtrain_val.csv")
train = trainentest.iloc[:273988,:]
test = trainentest.iloc[273988:, :]

In [2]:
trainentest[79670:80000]
starttime = trainentest['datetime_local'].at[79678]

startime = pd.to_datetime(starttime)


# Preprocessing the Data
### 1. Load features and get them into the base dataframe

- load all features into the base dataframe
- create base_df

### 2. Preprocessing of features
- create dummy
- normalization (DECISION: Min-Max Normalization) 
- one hot encoding
- interpolating (fill in in between values)

### 3. Cutting off the dataframe 
DECISION: ? 
Which datapoints to keep?
Pallet_movement_5min starts 2nd of Jan 2019, so first rows is NaN. 

-  Demand values we have:
12/31/2018  21:15:00  upandincluding 10/11/2021/ 6:07:00
-  Predicting the demand from 
10/11/2021/ 6:08 upandincluding 12/13/2021  17:59:00 

### 1. Load features and get them into the base dataframe
#### Load features

Total weight of pallets coming into the warehouse in the previous 1h, 5h, 10h, 23h

In [8]:
###Incoming weight feature preprocessing
#load all data
incoming_weight_1h = load_data('incoming_weight_df_1h.csv')
incoming_weight_5h = load_data('incoming_weight_df_5h.csv')
incoming_weight_10h = load_data('incoming_weight_df_10h.csv')
incoming_weight_23h = load_data('incoming_weight_df.csv')
#rename columns 
incoming_weight_5h = incoming_weight_5h.rename(columns = {'weight_23h' : 'weight_5h'})
incoming_weight_1h = incoming_weight_1h.rename(columns = {'weight_23h' : 'weight_1h'})
incoming_weight_10h = incoming_weight_10h.rename(columns = {'weight_23h' : 'weight_10h'})
#get them to a datetime object
incoming_weight_1h['datetime_local'] = pd.to_datetime(incoming_weight_1h['datetime_local'])
incoming_weight_5h['datetime_local'] = pd.to_datetime(incoming_weight_5h['datetime_local'])
incoming_weight_10h['datetime_local'] = pd.to_datetime(incoming_weight_10h['datetime_local'])
incoming_weight_23h['datetime_local'] = pd.to_datetime(incoming_weight_23h['datetime_local'])
#set index to be datetime
incoming_weight_1h.set_index('datetime_local', inplace=True)
incoming_weight_5h.set_index('datetime_local', inplace=True)
incoming_weight_10h.set_index('datetime_local', inplace=True)
incoming_weight_23h.set_index('datetime_local', inplace=True)
#reshape them to start at 2018-31-12 9:15PM
incoming_weight_1h = incoming_weight_1h[2361:]
incoming_weight_5h = incoming_weight_5h[2330:] 
incoming_weight_10h = incoming_weight_10h[2326:]
incoming_weight_23h = incoming_weight_23h[2323:]
#Drop duplicates
incoming_weight_1h = incoming_weight_1h.loc[~incoming_weight_1h.index.duplicated()]
incoming_weight_5h = incoming_weight_5h.loc[~incoming_weight_5h.index.duplicated()]
incoming_weight_10h = incoming_weight_10h.loc[~incoming_weight_10h.index.duplicated()]
incoming_weight_23h = incoming_weight_23h.loc[~incoming_weight_23h.index.duplicated()]


Total number of pallets moved within 5 minutes

In [102]:
#load data
pallet_move_5min = load_data('pallet_movement_5min_ft.csv')
#rename column
pallet_move_5min.rename(columns = {'quantity' : 'pallet_movement_5min'}, inplace = True)
#make index a datetime object and set as index
pallet_move_5min['datetime_local'] = pd.to_datetime(pallet_move_5min['datetime_local'])
pallet_move_5min.set_index('datetime_local', inplace = True)
#delete duplicates
pallet_move_5min = pallet_move_5min.loc[~pallet_move_5min.index.duplicated()]

In [103]:
base_door = load_data("feature_inbound_outbound_door_open.csv")
base_door = base_door.rename(columns = {'total' : 'doors_open'})
base_door['datetime_local'] = pd.to_datetime(base_door['datetime_local'])
base_door.set_index('datetime_local', inplace=True)

#### Create the base dataframe base_df

In [6]:
trainentest

,datetime_local,demand_kW
0,12/31/18 21:15,2064.101392
1,12/31/18 21:30,1874.002081
2,12/31/18 21:45,1988.168511
3,12/31/18 22:00,2022.795943
4,12/31/18 22:15,1986.981872
...,...,...
365344,12/13/21 17:55,NaN
365345,12/13/21 17:56,NaN
365346,12/13/21 17:57,NaN
365347,12/13/21 17:58,NaN


In [104]:
def addtimecol(df, colname): ####input df and colname 
    df[colname] = pd.to_datetime(df[colname])         
    df['year'] = df[colname].dt.year
    df['month'] = df[colname].dt.month
    df['weekday'] = df[colname].dt.weekday
    df['day'] = df[colname].dt.day
    df['hour'] = df[colname].dt.hour
    df['minute'] = df[colname].dt.minute        
    return df

#Create new dummy dfs
base_df = trainentest.copy()
base_weather = weather.copy()

#Remove unnecessary columns
base_df = base_df.drop('Unnamed: 0', axis=1)
base_weather = base_weather.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
base_weather["localstrptime"]= pd.to_datetime(base_weather["localstrptime"])
base_df['datetime_local'] = pd.to_datetime(base_df['datetime_local'])
base_weather = base_weather.rename(columns={'localstrptime':'datetime_local'})

#Add time columns.
addtimecol(base_df, 'datetime_local')

#Set index to datetime
base_df.set_index('datetime_local', inplace=True)
base_weather.set_index('datetime_local', inplace=True)

#Concatenate the weather DataFrame to the base DataFrame
base_df = pd.concat([base_df, base_weather], axis=1)

#Concatenate the incoming weight dataframe with the base dataframe
base_df = pd.concat([base_df, incoming_weight_1h], axis=1)
base_df = pd.concat([base_df, incoming_weight_5h], axis=1)
base_df = pd.concat([base_df, incoming_weight_10h], axis=1)
base_df = pd.concat([base_df, incoming_weight_23h], axis=1)

#Concatenate the pallet movement feature
base_df = pd.concat([base_df, pallet_move_5min], axis = 1)

#Concatenate the door feature
base_df = pd.concat([base_df, base_door], axis=1)

## Add values for the demand in the previous days at that moment of the day
##### timeseries_dummy only consists of the range of dates we have for which we have information per minute, because timeseries need to have regular intervals. 


In [149]:
### create timeseries_dummy 
starttime = trainentest['datetime_local'].at[79678]
startime = pd.to_datetime(starttime)
onlyminutesstep1 = base_df[:1460735]
onlyminutes = onlyminutesstep1.loc[starttime:]
timeseries_dummy = onlyminutes.copy()

### Create rows for missing minutes
timeseries_dummy = timeseries_dummy.asfreq(freq = 'T', method = 'pad')

### about 6900 random rows don't have demand, ffill is fill them with the previous amount
timeseries_dummy['demand_kW'] = timeseries_dummy.demand_kW.fillna(method = 'ffill', axis = 0)

### edit timeseries_dummy just as dummy_dataset
timeseries_dummy.reset_index(inplace = True)
timeseries_dummy.drop(['hour'], axis=1, inplace = True)
timeseries_dummy.drop(['datetime'], axis=1, inplace = True)
timeseries_dummy.drop(['datetime_local'], axis=1, inplace = True)
timeseries_dummy.drop(['datetime_UTC'], axis=1, inplace = True)

### setup a normalized dataframe
timeseries_normalized_dummy = timeseries_dummy.copy()

In [264]:
#A method to create 'lags' of a series; aka previous values in a cycle.  
#These values are added as columns to the timeseries_dummy
#The amount of columns that are added are the amount of lags
#The column used is in the list [col_names]
#The period of which the previous timepoint is taken are the amount of minutes before it, so cycle of a day is 60x24 minutes

def add_lags_of_column(df, col_names, lag = 3, period = 1):
    new_dict={}
    for col_name in col_names:
        new_dict[col_name]=df[col_name]
        # create lagged Series
        for l in range(1,lag+1):
            new_dict['%s_%amin_%d' %(col_name, period,l)]=df[col_name].shift(l*period)
    return pd.DataFrame(new_dict,index=df.index)


In [263]:
add_daycycle = add_lags_of_column(timeseries_dummy, ['demand_kW'], 10, 1440)
#add_hourcycle = add_lags_of_column(timeseries_dummy, ['demand_kW'], 10, 60)
#add_weekcycle = add_lags_of_column(timeseries_dummy, ['demand_kW'], 10, 10080)

pd.concat([timeseries_dummy, added_daycycle], axis = 1)

,demand_kW,year,month,weekday,day,minute,Relative Humidity,Temperature,weight_1h,weight_5h,...,demand_kW_1440min_1,demand_kW_1440min_2,demand_kW_1440min_3,demand_kW_1440min_4,demand_kW_1440min_5,demand_kW_1440min_6,demand_kW_1440min_7,demand_kW_1440min_8,demand_kW_1440min_9,demand_kW_1440min_10
0,1334.544,2021.0,5.0,0.0,24.0,4.0,NaN,NaN,88553.0,478724.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1302.755,2021.0,5.0,0.0,24.0,5.0,100.00,71.60,88553.0,478724.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1407.071,2021.0,5.0,0.0,24.0,6.0,NaN,NaN,88553.0,478724.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1328.769,2021.0,5.0,0.0,24.0,7.0,NaN,NaN,88553.0,478724.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1413.847,2021.0,5.0,0.0,24.0,8.0,NaN,NaN,88553.0,478724.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1511.455,2021.0,5.0,0.0,24.0,9.0,NaN,NaN,88553.0,478724.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1645.035,2021.0,5.0,0.0,24.0,10.0,100.00,71.60,88553.0,478724.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1727.209,2021.0,5.0,0.0,24.0,11.0,NaN,NaN,127163.0,517334.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1838.902,2021.0,5.0,0.0,24.0,12.0,NaN,NaN,127163.0,517334.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2090.810,2021.0,5.0,0.0,24.0,13.0,NaN,NaN,110174.0,517334.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2. Preprocessing of features

#### Creating the dummy df with which we will do model selection


In [201]:
dummy_df = base_df.copy()
dummy_df = dummy_df.reset_index()
dummy_df = dummy_df.drop(['hour'], axis=1)
dummy_df = dummy_df.drop(['datetime'], axis=1)
dummy_df = dummy_df.drop(['datetime_local'], axis=1)
dummy_df = dummy_df.drop(['datetime_UTC'], axis=1)
dummy_df

dummy_normalized_df = dummy_df.copy()
#Still drop demand_kW NaNs
dummy_normalized_df = dummy_normalized_df.dropna(subset=['demand_kW'], axis=0)

In [7]:
dummy_df.pallet_movement_5min.count()

1592126

#### Preprocessing functions defined
DECISION: MinMax Normalization

In [202]:
def normalize_column(df, columnname):
    """Function which returns a Dataframe where the given column is normalized through min-max normalization."""
    df[f'{columnname}_normalized'] = (df[columnname] - df[columnname].min()) / (df[columnname].max() - df[columnname].min())
    return df.drop([columnname], axis=1)

def add_one_hot_encoder(df, colname):
    """
    Function which returns a DataFrame where the given column has been removed and replaced by
    one-hot-encoding columns for each value in the original column.
    """
    onehot = pd.get_dummies(df[colname], prefix=colname)
    return df.drop(colname, axis=1).join(onehot)

def interpolate_column(df, colname):
    df[f'{colname}_interpolated'] = df[colname].interpolate(method='linear')
    return df.drop([colname], axis=1)

#### interpolate/one-hot-encoding/normalizing
DECISION: ?  normalization of the doors_open

In [203]:
interpolate = ['Temperature', 'Relative Humidity']
to_normalize = ['Relative Humidity_interpolated', 'Temperature_interpolated', 'weight_23h','weight_10h', 'weight_5h', 'weight_1h', 'pallet_movement_5min', 'doors_open']
add_one_hot_encoding = ['weekday', 'year', 'month']


for x in interpolate:
    dummy_normalized_df = interpolate_column(dummy_normalized_df, x)
    timeseries_normalized_dummy = interpolate_column(timeseries_normalized_dummy, x)
for x in to_normalize:
    dummy_normalized_df = normalize_column(dummy_normalized_df, x)
    timeseries_normalized_dummy = normalize_column(timeseries_normalized_dummy, x)
for x in add_one_hot_encoding:
    dummy_normalized_df = add_one_hot_encoder(dummy_normalized_df, x)
    timeseries_normalized_dummy = add_one_hot_encoder(timeseries_normalized_dummy, x)

In [204]:
timeseries_normalized_dummy

,demand_kW,day,minute,Relative Humidity_interpolated_normalized,Temperature_interpolated_normalized,weight_23h_normalized,weight_10h_normalized,weight_5h_normalized,weight_1h_normalized,pallet_movement_5min_normalized,...,weekday_4.0,weekday_5.0,weekday_6.0,year_2021.0,month_5.0,month_6.0,month_7.0,month_8.0,month_9.0,month_10.0
0,1334.544,24.0,4.0,NaN,NaN,0.454989,0.587159,0.556739,0.274719,0.172286,...,0,0,0,1,1,0,0,0,0,0
1,1302.755,24.0,5.0,1.000000,0.380228,0.454989,0.587159,0.556739,0.274719,0.158058,...,0,0,0,1,1,0,0,0,0,0
2,1407.071,24.0,6.0,1.000000,0.380228,0.454989,0.587159,0.556739,0.274719,0.171510,...,0,0,0,1,1,0,0,0,0,0
3,1328.769,24.0,7.0,1.000000,0.380228,0.454989,0.587159,0.556739,0.274719,0.151332,...,0,0,0,1,1,0,0,0,0,0
4,1413.847,24.0,8.0,1.000000,0.380228,0.454989,0.587159,0.556739,0.274719,0.098215,...,0,0,0,1,1,0,0,0,0,0
5,1511.455,24.0,9.0,1.000000,0.380228,0.454989,0.587159,0.556739,0.274719,0.149349,...,0,0,0,1,1,0,0,0,0,0
6,1645.035,24.0,10.0,1.000000,0.380228,0.454989,0.587159,0.556739,0.274719,0.146072,...,0,0,0,1,1,0,0,0,0,0
7,1727.209,24.0,11.0,1.000000,0.380228,0.474339,0.620881,0.601641,0.394500,0.125550,...,0,0,0,1,1,0,0,0,0,0
8,1838.902,24.0,12.0,1.000000,0.380228,0.474339,0.620881,0.601641,0.394500,0.121669,...,0,0,0,1,1,0,0,0,0,0
9,2090.810,24.0,13.0,1.000000,0.380228,0.474339,0.620881,0.601641,0.341794,0.128395,...,0,0,0,1,1,0,0,0,0,0




# Model selection and building
## 1. Importing model libraries

## 2. Divide Train and Test data

## 3. Test models 

In [11]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

## 2. Dividing train & test data

In [12]:
def test_model(model, data, debug=False):
    xtrain, ytrain, xtest, ytest = data
    if debug:
        print("Fitting model...")
    model = model.fit(xtrain, ytrain)
    if debug:
        print('Predicting...')
    acc_test = model.predict(xtest)
    acc_train = model.predict(xtrain)
    if debug:
        print('Calculating mean absolute error...')
    return [mean_absolute_error(list(ytrain), acc_train), mean_absolute_error(list(ytest), acc_test), model]

##### dummy_normalized_df['pallet_movement_5min'] drop NaN values

In [13]:
dummy_normalized_df = dummy_normalized_df.dropna(subset = ['pallet_movement_5min_normalized'])

In [14]:
#Define train, test sets\n",
train, test = train_test_split(dummy_normalized_df, shuffle=True)
X_train = train.copy().drop(['demand_kW'], axis=1)
Y_train = train['demand_kW']
X_test = test.copy().drop(['demand_kW'], axis=1)
Y_test = test['demand_kW']

#### Model Testing functions


In [15]:
parameter_settings = {
    'n_estimators': [20],
    'criterion': ['squared_error', 'friedman_mse', 'squared_error', 'poisson'],
    #'max_depth': [2, 4, 6],
    #'min_samples_split': [2, 4, 8]
}
data = [X_train, Y_train, X_test, Y_test]
model = RandomForestRegressor

In [16]:
def model_optimizer(data, model, params, splits, verbose=False):
    """
    
    """
    parameter_combinations = itertools.product(*params.values())
    results = {}
    for parameters in parameter_combinations:
        start = time.time()
        params_dict = dict(zip(params.keys(), parameters))
        print('Testing model with parameters: ' + str(params_dict))
        current_model = model(**params_dict)
        mae_train, mae_test, model = test_model(current_model, data, debug = False)
        print(f'Mean Absolute Error = {mae_test}')
        modelname = str(params_dict)
        end = time.time()
        print(f'Time spent: {end-start} seconds. \n')
        results[modelname] = [mae_test, (end-start), model]
    return results
        
#test = model_optimizer(data, RandomForestRegressor, ,  parameter_settings)

# Functions to help with model testing

In [17]:
from sklearn.model_selection import TimeSeriesSplit
import time

def run_timeseries_model(data, model, n_splits = 5, verbose=False):
    """
    Function which receives a data and a model as input.
    Can also receive the amount of splits you want as input.
    For debugging purposes, set verbose to True.
    
    Outputs a list of:
        - The resulting model
        - The final RMSE
        - A list of the resulting Train RMSEs (per fold)
        - A list of the resulting Test RMSEs (per fold)
    """
    
    ts = TimeSeriesSplit(n_splits=n_splits)
    count = 1
    train_rmses = []
    test_rmses = []
    for train, test in ts.split(data):
        if verbose:
            print(f'Starting fold {count}')
        cv_train, cv_test = data.iloc[train], data.iloc[test]
        y_train = cv_train['demand_kW']
        x_train = cv_train.drop(['demand_kW'], axis=1)
        y_test = cv_test['demand_kW']
        x_test = cv_test.drop(['demand_kW'], axis=1)
        train_start = time.time()
        if verbose:
            print("Training model...")
        model.fit(x_train, y_train)
        train_stop = time.time()
        
        predict_start = time.time()
        if verbose:
            print("Predicting...")
        y_pred_test = model.predict(x_test)
        y_pred_train = model.predict(x_train)
        predict_stop = time.time()
        
        rmse_start = time.time()
        if verbose:
            print("Calculating rmse's...")
        train_rmse = mean_squared_error(y_train, y_pred_train, squared=False)
        test_rmse = mean_squared_error(y_test, y_pred_test, squared=False)
        rmse_stop = time.time()
        
        train_rmses.append(train_rmse)
        test_rmses.append(test_rmse)
        timings = [(train_stop - train_start), (predict_stop - predict_start), (rmse_stop - rmse_start), (rmse_stop - train_start)]
        if verbose:
            print(f'Fold {count} train error: {train_rmse}. Test error: {test_rmse}. Time taken: {timings[3]} s.')
        count += 1
    all_rmses = [train_rmses, test_rmses]
    return [model, test_rmse, all_rmses, timings]

In [18]:
import itertools
import pickle

def find_optimal_parameters(data, model, params, n_splits=5, verbose=False):
    """
    Function which will find the optimal parameters within set boudaries.
    Receives as input:
    - The data
    - A model
    - The dictionary of parameters to optimize:
        - Key: parameter name
        - Value: List of testing parameter values.
    - The amount of Timeseries splits for crossvalidation (Standard: 5)
    - Whether information should be printed or not (Standard: False)
    """
    
    parameter_combinations = itertools.product(*params.values())
    results = {}
    count = 1
    for x in parameter_combinations:
        model_name = f'Model {count}'
        results[model_name] = [dict(zip(params.keys(), x))]
        count +=1
    lowest = 1000000000
    best_model = ""
    print(results['Model 1'])
    for model_name in results.keys():
        params_dict = results[model_name][0]
        print(f'Testing model {model_name} with parameters: {params_dict}')
        current_model = model(**params_dict)
        results[model_name].append(run_timeseries_model(data, current_model, n_splits=n_splits, verbose=verbose))
        print(results[model_name])
        print(f'Finished testing model. Test RMSE: {results[model_name][1][1]}. Time taken: {results[model_name][1][3][3]}.')
        if results[model_name][1][1] < lowest:
            lowest = results[model_name][1][1]
            best_model = model_name
    print(f"Optimization completed. Best model is {best_model} with parameters {results[model_name]}. \n RMSE is {results[model_name][1][1]}.")
    pickle.dump(best_model, open("Best_model.pkl", 'wb'))
    return results

In [19]:
find_optimal_parameters(dummy_normalized_df, RandomForestRegressor, parameter_settings, n_splits=2, verbose=True)

[{'n_estimators': 20, 'criterion': 'squared_error'}]
Testing model Model 1 with parameters: {'n_estimators': 20, 'criterion': 'squared_error'}
Starting fold 1
Training model...


KeyError: 'squared_error'

In [ ]:
parameter_settings_GB = {
    
    'n_estimators': [100],   
    'loss' : ['huber'],
    'learning_rate': [0.1, 0.3],
    'max_leaf_nodes': [None],
    'max_depth': [4, 8]
}


find_optimal_parameters(dummy_normalized_df, GradientBoostingRegressor, parameter_settings_GB, n_splits = 2, verbose = True)

In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))